In [3]:
# rmstools setup_wizard -dbpath /fs/cbsuhy02/storage/yc2553/yc2553/projects/TRE_directionality/test_DB/ -dbname test.db

DBPATH = "/fs/cbsuhy02/storage/yc2553/yc2553/projects/TRE_directionality/test_DB/"
DBNAME = "test.db"
DBRESDIR = "RMSResources/"
import sys
import logging
logger = logging.getLogger("rmspool")
logger.setLevel("INFO")
logger.addHandler(logging.StreamHandler(sys.stdout))
from rmsp import rmscore
rms = rmscore.ResourceManagementSystem(f"{DBPATH}{DBNAME}", f"{DBPATH}{DBRESDIR}")
from rmsp import rmsbuilder
rmspool = rmsbuilder.RMSProcessWrapPool(rms, 8)
rmsb = rmsbuilder.RMSUnrunTasksBuilder(rmspool)
from rmsp import rmsutils
# This ID is randomly generated to trace Jupyter files in RMS
rms.set_scriptID("bcc3719729074f69a93a31a4b58e9fda")

Checking tasks to skip...
Checking pending tasks to submit...
Total pending pids available to submit: 0 / 0


In [4]:
import pandas as pd
from commonhelper import load_func
from commandtemplate.conda import run_template_bash, OUTPUT_run_template_bash

In [3]:
conda_env = "DI"
pipeline = "./procapanalysis.py"

In [5]:
PROJECT_DIR_d = "/fs/cbsuhy02/storage/yc2553/yc2553/projects/TRE_directionality/PROcap/"
PROJECT_DIR_r = "/fs/cbsuhy02/storage/yc2553/yc2553/projects/TRE_directionality/resources/"
PROJECT_DIR_o = "/fs/cbsuhy02/storage/yc2553/yc2553/projects/TRE_directionality/output/"

# Pipeline registration

In [5]:
run_template_bash = rms.register_pipe(run_template_bash, output_func = OUTPUT_run_template_bash)

# Raw PRO-cap data

In [7]:
# We've deposited PRO-cap data in the ENCODE portal

procap_info = pd.read_excel(f"{PROJECT_DIR_o}tables/SuppTable1.xlsx")
procap_info

,Library Name,Internal Sample ID,Cell line,Treatment,S2 spike-in,5' adapter,3' adapter,Sequencing Read 1,Sequencing Read 2,ENCODE Experiment ID
0,C1a,C1a,K562,NaN,No,VRA5 + 6N,VRA3 + 6N,C1a_CKDL220028609-1A_HLJNNDSX5_L4_1.fq.gz,C1a_CKDL220028609-1A_HLJNNDSX5_L4_2.fq.gz,ENCSR220XSM
1,C1b,C1b,K562,NaN,No,VRA5 + 6N,VRA3 + 6N,C1b_CKDL220028609-1A_HLJNNDSX5_L4_1.fq.gz,C1b_CKDL220028609-1A_HLJNNDSX5_L4_2.fq.gz,ENCSR220XSM
2,CTCF_U1,CTCF_U1,HCT116,untreated,Yes,VRA5 + 6N,VRA3 + 6N,CTCF_U1_1.fq.gz,CTCF_U1_2.fq.gz,ENCSR165QKR
3,CTCF_U2,CTCF_U2,HCT116,untreated,Yes,VRA5 + 6N,VRA3 + 6N,CTCF_U2_1.fq.gz,CTCF_U2_2.fq.gz,ENCSR165QKR
4,CTCF_T1,CTCF_T1,HCT116,treated,Yes,VRA5 + 6N,VRA3 + 6N,CTCF_T1_1.fq.gz,CTCF_T1_2.fq.gz,ENCSR261OWE
5,CTCF_T2,CTCF_T2,HCT116,treated,Yes,VRA5 + 6N,VRA3 + 6N,CTCF_T2_1.fq.gz,CTCF_T2_2.fq.gz,ENCSR261OWE
6,ZNF143_U1,ZNF1,HEK293T,untreated,Yes,VRA5 + 6N,VRA3 + 6N,ZNF1_CKDL250028480-1A_22Y7T3LT4_L6_1.fq.gz,ZNF1_CKDL250028480-1A_22Y7T3LT4_L6_2.fq.gz,ENCSR473KVF
7,ZNF143_U2,ZNF2,HEK293T,untreated,Yes,VRA5 + 6N,VRA3 + 6N,ZNF2_CKDL250028480-1A_22Y7T3LT4_L6_1.fq.gz,ZNF2_CKDL250028480-1A_22Y7T3LT4_L6_2.fq.gz,ENCSR473KVF
8,ZNF143_T1,ZNF5,HEK293T,treated,Yes,VRA5 + 6N,VRA3 + 6N,ZNF5_CKDL250028480-1A_22Y7T3LT4_L6_1.fq.gz,ZNF5_CKDL250028480-1A_22Y7T3LT4_L6_2.fq.gz,ENCSR733DAS
9,ZNF143_T2,ZNF6,HEK293T,treated,Yes,VRA5 + 6N,VRA3 + 6N,ZNF6_CKDL250028480-1A_22Y7T3LT4_L6_1.fq.gz,ZNF6_CKDL250028480-1A_22Y7T3LT4_L6_2.fq.gz,ENCSR733DAS


In [7]:
files = []
for _, row in procap_info.iterrows():
	if type(row["Sequencing Read 1"]) == str:
		files.extend([row["Sequencing Read 1"], row["Sequencing Read 2"]])
for f in files:
	rms.register_file(f"{PROJECT_DIR_d}/RawData/{f}")

# Genome and related files

In [7]:
for f in [
	# https://www.encodeproject.org/files/GRCh38_no_alt_analysis_set_GCA_000001405.15/@@download/GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta.gz
	f"{PROJECT_DIR_r}genomes/human/GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta.gz", 
	# NCBI GenBank ID U13369.1; https://www.ncbi.nlm.nih.gov/nuccore/555853
	f"{PROJECT_DIR_r}genomes/human/rDNA_U13369.1.fasta",
	# https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.chrom.sizes
	f"{PROJECT_DIR_r}genomes/human/hg38.chrom.sizes.filtered",
	f"{PROJECT_DIR_r}genomes/human/hg38.chrom.sizes.filtered.no.chrY",
	# https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_37/gencode.v37.annotation.gtf.gz
	f"{PROJECT_DIR_r}genomes/human/gencode.v37.annotation.gtf.gz",
	# https://hgdownload.soe.ucsc.edu/goldenPath/dm6/bigZips/dm6.fa.gz
	f"{PROJECT_DIR_r}genomes/drosophila/dm6.fa.gz",
	# https://github.com/Boyle-Lab/Blacklist/raw/master/lists/hg38-blacklist.v2.bed.gz
	f"{PROJECT_DIR_r}genomes/human/hg38-blacklist.v2.bed.gz"
]:
	rms.register_file(f)

In [8]:
# Unzip files

f = lambda **kwargs: [kwargs['o']]
output_func = rms.register_pipe(f)
run_template_bash(
	"zcat {i} > {o}",
	output_func = output_func,
	i = rms.file_from_path(f"{PROJECT_DIR_r}genomes/human/GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta.gz"),
	o = f"{PROJECT_DIR_r}genomes/human/GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta"
)
run_template_bash(
	"zcat {i} > {o}",
	output_func = output_func,
	i = rms.file_from_path(f"{PROJECT_DIR_r}genomes/human/gencode.v37.annotation.gtf.gz"),
	o = f"{PROJECT_DIR_r}genomes/human/gencode.v37.annotation.gtf"
)
run_template_bash(
	"zcat {i} > {o}",
	output_func = output_func,
	i = rms.file_from_path(f"{PROJECT_DIR_r}genomes/drosophila/dm6.fa.gz"),
	o = f"{PROJECT_DIR_r}genomes/drosophila/dm6.fa"
)

In [10]:
# Add prefix to dm6 for spike-in indication

f = lambda **kwargs: [kwargs['o']]
output_func = rms.register_pipe(f)
run_template_bash(
	"biodatatools modify_fasta_names -i {i} -o {o} -func {func}",
	conda_env = conda_env,
	output_func = output_func,
	i = rms.file_from_path(f"{PROJECT_DIR_r}genomes/drosophila/dm6.fa"),
	o = f"{PROJECT_DIR_r}genomes/drosophila/dm6_addprefix.fa",
	func = "'lambda x: \"dm6_\" + x'"
)

# STAR genome index

## Human without spikein

In [13]:
# C1 library: no spike-in

f = lambda **kwargs: [kwargs['genomeDir']]
output_func = rms.register_pipe(f)
run_template_bash(
	"STAR --runMode genomeGenerate --readFilesCommand zcat --runThreadN {thread} --genomeDir {genomeDir} --genomeFastaFiles {i0} {i1} --sjdbGTFfile {annotation_file}",
	conda_env = conda_env,
	output_func = output_func,
	i0 = rms.file_from_path(f"{PROJECT_DIR_r}genomes/human/GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta"),
	i1 = rms.file_from_path(f"{PROJECT_DIR_r}genomes/human/rDNA_U13369.1.fasta"),
	annotation_file = rms.file_from_path(f"{PROJECT_DIR_r}genomes/human/gencode.v37.annotation.gtf"),
	thread = 32,
	genomeDir = f"{PROJECT_DIR_r}STAR_genome_index/hg38_rDNA"
	)

!!!!! WARNING: Could not move Log.out file from ./Log.out into /fs/cbsuhy02/storage/yc2553/yc2553/projects/TRE_directionality/resources/STAR_genome_index/hg38_rDNA/Log.out. Will keep ./Log.out




	/home/yc2553/miniconda3_new/envs/DI_test/bin/STAR-avx2 --runMode genomeGenerate --readFilesCommand zcat --runThreadN 32 --genomeDir /fs/cbsuhy02/storage/yc2553/yc2553/projects/TRE_directionality/resources/STAR_genome_index/hg38_rDNA --genomeFastaFiles /fs/cbsuhy02/storage/yc2553/yc2553/projects/TRE_directionality/resources/genomes/human/GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta /fs/cbsuhy02/storage/yc2553/yc2553/projects/TRE_directionality/resources/genomes/human/rDNA_U13369.1.fasta --sjdbGTFfile /fs/cbsuhy02/storage/yc2553/yc2553/projects/TRE_directionality/resources/genomes/human/gencode.v37.annotation.gtf
	STAR version: 2.7.11a   compiled: 2023-09-15T02:58:53+0000 :/opt/conda/conda-bld/star_1694746407721/work/source
Oct 11 17:02:53 ..... started STAR run
Oct 11 17:02:53 ... starting to generate Genome files
Oct 11 17:04:28 ..... processing annotations GTF
Oct 11 17:05:28 ... starting to sort Suffix Array. This may take a long time...
Oct 11 17:06:02 ... sorting Suffix Array

## Human with drosophila spikein

In [ ]:
# Degron lines: with spike-in

f = lambda **kwargs: [kwargs['genomeDir']]
output_func = rms.register_pipe(f)
run_template_bash(
	"STAR --runMode genomeGenerate --readFilesCommand zcat --runThreadN {thread} --genomeDir {genomeDir} --genomeFastaFiles {i0} {i1} {i2} --sjdbGTFfile {annotation_file}",
	conda_env = conda_env,
	output_func = output_func,
	i0 = rms.file_from_path(f"{PROJECT_DIR_r}genomes/human/GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta"),
	i1 = rms.file_from_path(f"{PROJECT_DIR_r}genomes/human/rDNA_U13369.1.fasta"),
	i2 = rms.file_from_path(f"{PROJECT_DIR_r}genomes/drosophila/dm6_addprefix.fa"),
	annotation_file = rms.file_from_path(f"{PROJECT_DIR_r}genomes/human/gencode.v37.annotation.gtf"),
	thread = 32,
	genomeDir = f"{PROJECT_DIR_r}STAR_genome_index/hg38_rDNA_dm6"
)

# GENCODE annotation 

In [15]:
# This file is used to classify distal vs. proximal elements

f = lambda **kwargs: [kwargs['o']]
output_func = rms.register_pipe(f)
run_template_bash(
	"biodatatools generate_union_TSS -i {i} -o {o} -forward_len {forward_len} -reverse_len {reverse_len}",
	conda_env = conda_env,
	output_func = output_func,
	i = rms.file_from_path(f"{PROJECT_DIR_r}genomes/human/gencode.v37.annotation.gtf.gz"),
	o = f"{PROJECT_DIR_r}genomes/human/gencode.v37.annotation.TSS.500.bed.bgz",
	forward_len = 500,
	reverse_len = 500
)

In [16]:
# This file is used to classify intergenic vs. intragenic elements

f = lambda **kwargs: [kwargs['o']]
output_func = rms.register_pipe(f)
run_template_bash(
	"biodatatools generate_union_transcripts -i {i} -o {o}",
	conda_env = conda_env,
	output_func = output_func,
	i = rms.file_from_path(f"{PROJECT_DIR_r}genomes/human/gencode.v37.annotation.gtf.gz"),
	o = f"{PROJECT_DIR_r}genomes/human/gencode.v37.annotation.transcripts.union.bed.bgz"
)

In [17]:
f = lambda **kwargs: [kwargs['o']]
output_func = rms.register_pipe(f)
run_template_bash(
	"biodatatools generate_geneannotations_union_features -i {i} -o {o} -feature {f}",
	conda_env=conda_env,
	output_func=output_func,
	i = rms.file_from_path(f"{PROJECT_DIR_r}genomes/human/gencode.v37.annotation.gtf.gz"),
	o = f"{PROJECT_DIR_r}genomes/human/gencode.v37.annotation.CDS.union.bed.bgz",
	f = "CDS"
)

In [23]:
# Exclude coding or blacklist regions
# Used to generate control regions later

f = lambda **kwargs: [kwargs['o']]
output_func = rms.register_pipe(f)
run_template_bash(
	"zcat {i0} {i1} | cut -f 1-3 | gzip > {o}",
	conda_env=conda_env,
	output_func=output_func,
	i0 = rms.file_from_path(f"{PROJECT_DIR_r}genomes/human/gencode.v37.annotation.CDS.union.bed.bgz"),
	i1 = rms.file_from_path(f"{PROJECT_DIR_r}genomes/human/hg38-blacklist.v2.bed.gz"),
	o = f"{PROJECT_DIR_r}genomes/human/excluded_regions.bed.gz",
)

# Generate annotation for QC (gene body ratio)

In [19]:
RMSTemplate_generate_QC_gbratio_annotation_20240901 = load_func(pipeline, "RMSTemplate_generate_QC_gbratio_annotation_20240901")

In [20]:
RMSTemplate_generate_QC_gbratio_annotation_20240901(
	rms,
	conda_env = conda_env,
	annotation_file = f"{PROJECT_DIR_r}genomes/human/gencode.v37.annotation.gtf.gz",
	o = f"{PROJECT_DIR_d}QC/Annotations/gencode.v37.annotation_proteincoding_nonoverlapping_long_transcripts.gtf.gz",
	discarded_chroms = ["chrY"]
)